In [1]:
#import numpy as np
import pandas as pd
title = ["name","landmass","zone","area","population","language","religion","bars","stripes","colours","red","green","blue","gold","white","black","orange","mainhue","circles","crosses",\
         "saltires","quarters","sunstars","crescent","triangle","icon","animate","text","topleft","botright"]
data = pd.read_csv("flag.data",skiprows=1,names=title, header=None)

In [2]:
# setup pyspark
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, asc

conf = (
    SparkConf()
    .setAppName('Spark Demo!')
)

conf.set("spark.hadoop.fs.s3a.endpoint", os.getenv('STORAGE_URL')) \
    .set('fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider') \
    .set('fs.s3a.access.key', f'{STS_ACCESSKEY}') \
    .set('fs.s3a.secret.key', f'{STS_SECRETKEY}') \
    .set('fs.s3a.session.token', f'{STS_SESSIONTOKEN}') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.multipart.size", "128M") \
    .set("spark.hadoop.fs.s3a.fast.upload.active.blocks", 8) \
    .set("spark.jars", "/home/jovyan/work/jars/hadoop-aws-3.2.0.jar,/home/jovyan/work/jars/aws-java-sdk-bundle-1.11.908.jar")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

##======================================================
#from pyspark.sql import SparkSession
#Create PySpark SparkSession
#spark = SparkSession.builder.getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(data) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- name: string (nullable = true)
 |-- landmass: long (nullable = true)
 |-- zone: long (nullable = true)
 |-- area: long (nullable = true)
 |-- population: long (nullable = true)
 |-- language: long (nullable = true)
 |-- religion: long (nullable = true)
 |-- bars: long (nullable = true)
 |-- stripes: long (nullable = true)
 |-- colours: long (nullable = true)
 |-- red: long (nullable = true)
 |-- green: long (nullable = true)
 |-- blue: long (nullable = true)
 |-- gold: long (nullable = true)
 |-- white: long (nullable = true)
 |-- black: long (nullable = true)
 |-- orange: long (nullable = true)
 |-- mainhue: string (nullable = true)
 |-- circles: long (nullable = true)
 |-- crosses: long (nullable = true)
 |-- saltires: long (nullable = true)
 |-- quarters: long (nullable = true)
 |-- sunstars: long (nullable = true)
 |-- crescent: long (nullable = true)
 |-- triangle: long (nullable = true)
 |-- icon: long (nullable = true)
 |-- animate: long (nullable = true)
 |-- text: lo

In [3]:
print((sparkDF.count(), len(sparkDF.columns)))

(193, 30)


In [4]:
#data.write.parquet(" 's3a://abcd/int/test/car_data.parquet")
path = "s3a://abcd/int/data/flag.parquet"
# df.write.parquet(path,mode="overwrite")
# reduce amount of partitions .coalesce(1)
sparkDF.write.parquet(path,mode="overwrite")

In [5]:
sparkDF2 = spark.read.format("parquet").load("s3a://abcd/int/data/flag.parquet")
sparkDF2.show()

+---------------+--------+----+----+----------+--------+--------+----+-------+-------+---+-----+----+----+-----+-----+------+-------+-------+-------+--------+--------+--------+--------+--------+----+-------+----+-------+--------+
|           name|landmass|zone|area|population|language|religion|bars|stripes|colours|red|green|blue|gold|white|black|orange|mainhue|circles|crosses|saltires|quarters|sunstars|crescent|triangle|icon|animate|text|topleft|botright|
+---------------+--------+----+----+----------+--------+--------+----+-------+-------+---+-----+----+----+-----+-----+------+-------+-------+-------+--------+--------+--------+--------+--------+----+-------+----+-------+--------+
|        Albania|       3|   1|  29|         3|       6|       6|   0|      0|      3|  1|    0|   0|   1|    0|    1|     0|    red|      0|      0|       0|       0|       1|       0|       0|   0|      1|   0|    red|     red|
|        Algeria|       4|   1|2388|        20|       8|       2|   2|      0|  